Controlador MRAC aplicado a uma planta de segunda ordem, com grau relativo 1

In [10]:
#all imports
import numpy as np 
from numpy import linalg as LA
import matplotlib.pyplot as plt
from scipy import signal

In [11]:
# system variables
Ap, Bp, Cp = np.matrix([[0.8,0.8],[0,0.8]]), np.matrix([1, 1]), np.matrix([1, 1])
Am, Bm, Cm = np.matrix([[0.2,0],[0,0.2]]), np.matrix([0.2, 0.2]), np.matrix([1, 1])

In [12]:
# learning rate
gamma = 100
Mtheta = 0.014

# simulation configs
n_points = 500000
t_f = 20

In [13]:
# sempre theta1, theta2, thetay, thetar
theta = np.zeros((n_points, 4))
theta[0] = (0.001, 0.001, 0.001, 0.001)
# sempre omega1, omega2, omegay, omegar 
omega = np.zeros((n_points, 4))

# sys plant x_m
x_m = np.zeros((n_points, 2))
y_m = np.zeros((n_points, 1))
# sys model x_p, y_p
x_p = np.zeros((n_points, 2))
# x_p[0] = [0.1,0.1]
y_p = np.zeros((n_points, 1))
# y_p[0] = 0.1

# Comecando sempre w1, w2, ... , wn
filtered_signal = np.zeros((n_points, 2))
# error
error = np.zeros((n_points, 1))

In [14]:
# signal
T, step = np.linspace(0, t_f, n_points, retstep=True)
print("step: ", step)
r = np.sin(0.2*np.pi*T)    #sinusoidal signal
# r = signal.square(0.2 * np.pi * T) #square signal
# r = signal.sawtooth(0.2 * np.pi * T) #sawtooth signal

# control signal
u = np.zeros((n_points, 1))
norm = np.zeros((n_points, 1))
sigma = np.zeros((n_points, 1))

step:  4.000008000016e-05


In [15]:
F = 1
q = 1
Fd = 1-F*step
qd=q*step

In [16]:
k = 10
norm_theta = LA.norm(theta[k])
sigmaeq = -gamma*error[k]*u[k]/(norm_theta**2)
norm_theta

C:\Users\queiroz\AppData\Local\Temp/ipykernel_3064/3485988466.py:3: RuntimeWarning: invalid value encountered in true_divide
  sigmaeq = -gamma*error[k]*u[k]/(norm_theta**2)


0.0

In [17]:
# main loop
for k, _ in enumerate(T):
  if(k < n_points - 1):
    # planta
    x_p[k + 1] = Ap.dot(x_p[k]) + Bp.dot(float(u[k]))
    y_p[k + 1] = Cp.dot(x_p[k])

    # ref
    x_m[k + 1] = Am.dot(x_m[k]) + Bm.dot(r[k])
    y_m[k + 1] = Cm.dot(x_m[k])

    # error 
    error[k] = y_p[k + 1] - y_m[k + 1]
    theta[k + 1] = theta[k] - step*gamma*error[k]*omega[k]

    # # omega é definido w1, w2, y, r
    omega[k + 1][0] = omega[k][0]*Fd + qd*u[k]
    omega[k + 1][1] = omega[k][1]*Fd + qd*y_m[k]
    omega[k + 1][2] = float(y_m[k])
    omega[k + 1][3] = r[k]

    u[k + 1] = (theta[k].transpose()).dot(omega[k])
    
    

In [ ]:
#plots
%matplotlib qt
plt.rcParams['figure.figsize'] = [8, 5]
fig, axs = plt.subplots(3)
fig.subplots_adjust(hspace=0.65)
axs[0].plot(T, y_p, label="y_p")
axs[0].plot(T, y_m, '--', label="y_m")
axs[0].legend()
axs[0].set_title('Performance')

axs[2].set_title('Erro da adaptação')
axs[2].plot(T, error, label='Error')
axs[2].legend()

axs[1].set_title('Adaptação dos regressores')
axs[1].plot(T, theta[:, 0], label='theta_w')
axs[1].plot(T, theta[:, 1], label='theta_r')
axs[1].plot(T, theta[:, 2], label='theta_1')
axs[1].plot(T, theta[:, 3], label='theta_2')
axs[1].legend()
axs[0].grid()
axs[1].grid()
axs[2].grid()
# axs[3].plot(T, norm, label="norma")
# axs[3].legend()

plt.show()